In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
int_df_main = pd.read_csv('drive/My Drive/int_df_main.csv')
int_df = pd.read_csv('drive/My Drive/int_df.csv')

In [3]:
euro_sched = pd.read_csv('drive/My Drive/uefa-euro-2024-UTC.csv')
euro_groupstage = euro_sched[:36]

In [5]:
euro_groupstage =  euro_groupstage.replace({'Czechia': 'Czech Republic', 'Türkiye': 'Turkey',})
euro_groupstage.columns = [c.replace(' ', '_') for c in euro_sched.columns]

In [ ]:
euro_groupstage

,Match_Number,Round_Number,Date,Location,Home_Team,Away_Team,Group,Result
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,5 - 1
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,1 - 3
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,3 - 0
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,2 - 1
4,5,1,16/06/2024 13:00,Volksparkstadion,Poland,Netherlands,Group D,1 - 2
5,6,1,16/06/2024 16:00,Arena Stuttgart,Slovenia,Denmark,Group C,1 - 1
6,7,1,16/06/2024 19:00,Arena AufSchalke,Serbia,England,Group C,0 - 1
7,8,1,17/06/2024 13:00,Fußball Arena München,Romania,Ukraine,Group E,3 - 0
8,9,1,17/06/2024 16:00,Frankfurt Stadion,Belgium,Slovakia,Group E,0 - 1
9,10,1,17/06/2024 19:00,Düsseldorf Arena,Austria,France,Group D,0 - 1


In [6]:
euro_groupstageR1 = euro_groupstage[0:12]
euro_groupstageR2 = euro_groupstage[12:24]
euro_groupstageR3 = euro_groupstage[24:36]

In [7]:
import pandas as pd

def create_fixture_dataframe(euro_groupstage, int_df, home_team_col='Home_Team', away_team_col='Away_Team'):
    # Get unique home teams from the group stage
    euro_teamsv1 = euro_groupstage[home_team_col].unique()
    test1 = euro_teamsv1.tolist()
    home_teams_df = pd.DataFrame(test1, columns=['HomeTeam'])

    # Get the last match information for each team
    test2 = int_df.groupby(['team'], as_index=False).last()

    # Filter teams that are in the list of home teams
    Euro_teams = test2[test2.team.isin(test1)]

    # Merge home teams with their last match information
    mergedf = home_teams_df.merge(Euro_teams, how='left', left_on='HomeTeam', right_on='team')

    # Get unique away teams from the group stage
    euro_teamsaway = euro_groupstage[away_team_col].unique()
    test1away = euro_teamsaway.tolist()
    away_teams_df = pd.DataFrame(test1away, columns=['AwayTeam'])

    # Filter teams that are in the list of away teams
    Euro_teams_away = test2[test2.team.isin(test1away)]

    # Merge away teams with their last match information
    mergedf_away = away_teams_df.merge(Euro_teams_away, how='left', left_on='AwayTeam', right_on='team')

    # Combine home and away teams dataframes
    df_ready = pd.concat([mergedf, mergedf_away], axis=1)

    return df_ready

# Example usage for each group stage round
df_readyR1 = create_fixture_dataframe(euro_groupstageR1, int_df)
df_readyR2 = create_fixture_dataframe(euro_groupstageR2, int_df)
df_readyR3 = create_fixture_dataframe(euro_groupstageR3, int_df)


In [8]:
df_ready = pd.concat([df_readyR1, df_readyR2, df_readyR3], axis=0, ignore_index=True)

df_ready

,HomeTeam,team,Unnamed: 0,GameId,date,scored,conceded,rank,total_points,points_by_rank,...,points_by_rank,is_stake,scored_rolling,conceded_rolling,rank_rolling,points_by_rank_rolling,scored_ewm,conceded_ewm,rank_ewm,points_by_rank_ewm
0,Germany,Germany,36964,18483,2024-03-26,2.0,1.0,16.0,1631.22,271.870000,...,20.363378,0,1.6,1.9,33.8,181.488503,1.541673,1.985258,35.777929,156.654391
1,Hungary,Hungary,36967,18484,2024-03-26,2.0,0.0,27.0,1525.13,14.952255,...,26.023226,0,1.7,1.1,13.7,26.056653,1.460937,1.174916,14.346189,87.450849
2,Spain,Spain,36988,18495,2024-03-26,3.0,3.0,8.0,1732.64,346.528000,...,47.710278,1,1.6,0.5,7.5,70.042860,1.319211,0.594023,9.135911,125.498622
3,Italy,Italy,36872,18437,2024-03-24,2.0,0.0,9.0,1718.82,55.445806,...,53.180385,0,1.4,0.7,63.8,27.607357,1.156226,0.933375,63.718158,34.849077
4,Poland,Poland,36911,18456,2024-03-26,0.0,0.0,30.0,1520.24,52.422069,...,109.092500,0,2.5,1.0,6.6,153.557968,2.515275,0.869485,7.239705,169.422282
5,Slovenia,Slovenia,36987,18494,2024-03-26,2.0,0.0,55.0,1431.40,204.485714,...,11.861556,0,1.6,0.9,18.9,26.291083,1.443653,0.881421,17.017686,63.397169
6,Serbia,Serbia,36893,18447,2024-03-25,1.0,0.0,32.0,1517.43,12.139440,...,450.012500,0,2.4,0.5,4.2,90.484194,1.966919,0.689231,4.219160,126.514156
7,Romania,Romania,36953,18477,2024-03-26,2.0,3.0,45.0,1472.73,105.195000,...,21.278767,1,1.6,1.2,25.8,125.342193,1.576558,0.965658,25.375103,96.045217
8,Belgium,Belgium,36960,18481,2024-03-26,2.0,2.0,4.0,1798.46,599.486667,...,31.863696,0,1.7,1.0,49.4,52.310922,1.492637,1.118095,48.633673,45.570610
9,Austria,Austria,36951,18476,2024-03-26,6.0,1.0,25.0,1546.10,44.174286,...,43.939048,0,3.0,0.8,2.1,72.355476,3.041997,1.010850,2.454731,112.997939


In [9]:
df_ready.to_csv('drive/My Drive/df_ready.csv')
df_ready = pd.read_csv('drive/My Drive/df_ready.csv')

In [10]:
df_ready = df_ready.filter(['HomeTeam', 'scored_rolling', 'conceded_rolling', 'rank_rolling', 'points_by_rank_rolling',
                            'scored_ewm', 'conceded_ewm', 'rank_ewm', 'points_by_rank_ewm','is_stake',
                        'AwayTeam', 'scored_rolling.1', 'conceded_rolling.1',
                        'rank_rolling.1', 'points_by_rank_rolling.1',
                            'scored_ewm.1', 'conceded_ewm.1', 'rank_ewm.1', 'points_by_rank_ewm.1',], axis=1)

In [11]:
df_ready = df_ready.rename(columns={'HomeTeam': 'HomeTeam', 'scored_rolling': 'HomeScored_rolling',
                                    'conceded_rolling': 'HomeConceded_rolling',
                        'rank_rolling': 'HomeRank_rolling', 'points_by_rank_rolling': 'HomePoints_by_rank_rolling',
                        'scored_ewm': 'HomeScored_ewm', 'conceded_ewm': 'HomeConceded_ewm',
                        'rank_ewm': 'HomeRank_ewm', 'points_by_rank_ewm': 'HomePoints_by_rank_ewm',
                        'is_stake': 'is_stake',
                        'AwayTeam': 'AwayTeam',
                        'scored_rolling.1': 'AwayScored_rolling',
                        'conceded_rolling.1': 'AwayConceded_rolling',
                        'rank_rolling.1': 'AwayRank_rolling', 'points_by_rank_rolling.1': 'AwayPoints_by_rank_rolling',
                        'scored_ewm.1': 'AwayScored_ewm', 'conceded_ewm.1': 'AwayConceded_ewm',
                        'rank_ewm.1': 'AwayRank_ewm', 'points_by_rank_ewm.1': 'AwayPoints_by_rank_ewm',})

In [12]:
df_ready.is_stake = 1

In [13]:
#setja upp features og spá.
df_ready['rank_difference_rolling'] = (df_ready['HomeRank_rolling'] - df_ready['AwayRank_rolling']) * -1
df_ready['score_difference_rolling'] = df_ready['HomeScored_rolling'] - df_ready['AwayScored_rolling']
df_ready['conceded_difference_rolling'] = (df_ready['HomeConceded_rolling'] - df_ready['AwayConceded_rolling'])
df_ready['Points_by_rank_rolling_difference'] = df_ready['HomePoints_by_rank_rolling'] - df_ready['AwayPoints_by_rank_rolling']
df_ready['rank_difference_ewm'] = (df_ready['HomeRank_ewm'] - df_ready['AwayRank_ewm']) * -1
df_ready['score_difference_ewm'] = df_ready['HomeScored_ewm'] - df_ready['AwayScored_ewm']
df_ready['conceded_difference_ewm'] = (df_ready['HomeConceded_ewm'] - df_ready['AwayConceded_ewm'])
df_ready['Points_by_rank_rolling_ewm'] = df_ready['HomePoints_by_rank_ewm'] - df_ready['AwayPoints_by_rank_ewm']

In [14]:
fixturesEuro2024 = df_ready.filter(['HomeTeam', 'AwayTeam'])
X_df = df_ready.filter(['rank_difference_rolling', 'score_difference_rolling', 'conceded_difference_rolling',
                        'Points_by_rank_rolling_difference',
                        'rank_difference_ewm', 'score_difference_ewm', 'conceded_difference_ewm',
                        'Points_by_rank_rolling_ewm'])

In [15]:
X_df_best = df_ready.filter(['rank_difference_rolling', 'rank_difference_ewm',
       'score_difference_ewm', 'conceded_difference_ewm'])

In [ ]:
#sækjum módelið
import pickle

pick_read = open('drive/My Drive/model_salka_Euros.pickle','rb')
model_salka_Euros = pickle.load(pick_read)
pick_read.close()

In [ ]:
#sækjum besta módelið
import pickle

pick_read = open('drive/My Drive/model_salka_Euros_best.pickle','rb')
model_salka_Euros_best = pickle.load(pick_read)
pick_read.close()

In [ ]:
#sækjum xgboost
import pickle

pick_read = open('drive/My Drive/model_salka_Euros_xgboost.pickle','rb')
model_salka_xgboost = pickle.load(pick_read)
pick_read.close()

In [22]:
#sækjum ensemble
import pickle

pick_read = open('drive/My Drive/model_salka_Euros_ensemble.pickle','rb')
model_salka_ensemble = pickle.load(pick_read)
pick_read.close()

In [23]:
predsgroupstage = model_salka_ensemble.predict_proba(X_df)
predsgroupstage = pd.DataFrame(data=predsgroupstage)

In [24]:
SalkaSpa_groupstage = pd.concat([euro_groupstage, predsgroupstage], axis=1)

In [25]:
SalkaSpa_groupstage = SalkaSpa_groupstage.drop(["Match_Number", "Round_Number",
                              'Date', 'Location', 'Group',
                              'Result'], axis = 1)

In [26]:
SalkaSpa_groupstage = SalkaSpa_groupstage.rename(columns={0: 'HomeWinProb', 1: 'DrawProb', 2: 'AwayWinProb'})

In [28]:
SalkaSpa_groupstage

,Home_Team,Away_Team,HomeWinProb,DrawProb,AwayWinProb
0,Germany,Scotland,0.476939,0.285625,0.237436
1,Hungary,Switzerland,0.321261,0.318087,0.360652
2,Spain,Croatia,0.437720,0.267361,0.294918
3,Italy,Albania,0.483048,0.250245,0.266708
4,Poland,Netherlands,0.257639,0.275653,0.466708
5,Slovenia,Denmark,0.285533,0.330835,0.383633
6,Serbia,England,0.280706,0.289559,0.429734
7,Romania,Ukraine,0.295021,0.273870,0.431110
8,Belgium,Slovakia,0.500589,0.291242,0.208169
9,Austria,France,0.250395,0.351635,0.397971


In [ ]:
SalkaSpa_groupstage.to_csv('drive/My Drive/SalkaSpa_groupstage.csv')